<a id='0.1'></a>
## 0.1 load required packages

In [1]:
%run "..\..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# library design specific tools
from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools as lt
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pandas as pd

22100


E:\Users\puzheng\Documents\ImageAnalysis3\library_tools\sequences.py:370: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if infotype is 'input' or infotype is 'all':
E:\Users\puzheng\Documents\ImageAnalysis3\library_tools\sequences.py:370: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if infotype is 'input' or infotype is 'all':
E:\Users\puzheng\Documents\ImageAnalysis3\library_tools\references.py:32: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if infotype is 'input' or infotype is 'all':
E:\Users\puzheng\Documents\ImageAnalysis3\library_tools\references.py:32: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if infotype is 'input' or infotype is 'all':


<a id='0.2'></a>
## 0.2 folders

In [3]:
## Some folders
# human genome
reference_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl'
genome_folder = os.path.join(reference_folder, 'Genome')
# Library directories
pool_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers'
# folder for sub-pool
library_folder = os.path.join(pool_folder, f'mop_SE_1000')

In [4]:
# laod encoding
# summarize total readout usage
encoding_folder = os.path.join(pool_folder, f'Encoding_design')
region_2_bits = pickle.load(open(os.path.join(encoding_folder, 'region_2_bits.pkl'), 'rb'))

In [5]:
# load probes
probes = []
final_pb_filename = os.path.join(library_folder, 'blast_full_probes_comb_plus_seq.fasta')

with open(final_pb_filename, 'r') as handle:
    for record in SeqIO.parse(handle, "fasta"):
        probes.append(record)

In [6]:
# load region names and real coordinates
region_2_names = {}
region_2_coords = {}
for _pb in probes:
    _reg = _pb.id.split('loc_')[1].split('_')[0]
    _start, _end = _reg.split(':')[1].split('-')
    # gene info
    _gene = _pb.id.split('gene_')[1].split('_')[0]
    _pb_pos = int(_pb.id.split('pos_')[1].split('_')[0]) + int(_start)
    
    if _gene not in region_2_names:
        region_2_names[_gene] = _reg
    if _gene not in region_2_coords:
        region_2_coords[_gene] = [_pb_pos]
    else:
        region_2_coords[_gene].append(_pb_pos)
print(region_2_names)

{'251': '4:105899094-105952080', '252': '4:114589240-114676242', '253': '4:115037700-115112192', '255': '4:115837369-115913327', '256': '4:116628906-116683403', '257': '4:117618081-117687368', '258': '4:123199825-123264995', '259': '4:123961238-124063077', '260': '4:124824301-124866678', '261': '4:126207621-126263017', '262': '4:126995025-127049608', '263': '4:127493306-127563000', '264': '4:127666510-127718932', '265': '4:127800701-127872488', '266': '4:129485757-129532804', '267': '4:130236140-130307752', '268': '4:130509810-130579025', '269': '4:131733908-131844723', '270': '4:132792079-132834284', '271': '4:135807186-135873467', '272': '4:137277244-137324456', '273': '4:148835951-148933937', '274': '4:155759707-155802098', '275': '5:3994952-4090244', '277': '5:4937946-5030393', '278': '5:5216972-5313712', '279': '5:15947115-16026399', '280': '5:19894500-19961425', '281': '5:21457453-21528145', '282': '5:24770084-24831106', '283': '5:25594458-25634378', '284': '5:27049822-27109916',

In [7]:
# generate dict
pb_len = 42
min_probe_num = 50

region_2_probeInfo = {}
for _gene in region_2_coords:
    _name = region_2_names[_gene]
    _chr = _name.split(':')[0]
    _coords = region_2_coords[_gene]
    if len(_coords) < min_probe_num:
        print(_name)
        continue
    _info = f"{_chr}:{np.min(_coords)}-{np.max(_coords)+pb_len}"
    region_2_probeInfo[_gene] = _info

13:114065922-114149764
13:114065922-114149495


In [8]:
# laod encoding
# summarize total readout usage
region_2_readout_dict = pickle.load(open(os.path.join(library_folder, 
                                                      'region_2_readout.pkl'), 'rb'))
print(region_2_readout_dict)
print(len(region_2_readout_dict))

{1: ['c3', 'c47', 'c76'], 2: ['c8', 'c54', 'c90'], 3: ['c29', 'c69', 'c87'], 4: ['c51', 'c65', 'c73'], 5: ['c35', 'c53', 'c60'], 6: ['c11', 'c18', 'c19'], 7: ['c52', 'c62', 'c89'], 8: ['c14', 'c30', 'c85'], 9: ['c15', 'c63', 'c84'], 10: ['c43', 'c68', 'c70'], 11: ['c45', 'c57', 'c81'], 12: ['c12', 'c79', 'c92'], 13: ['c42', 'c47', 'c66'], 14: ['c20', 'c21', 'c39'], 15: ['c9', 'c71', 'c72'], 16: ['c17', 'c31', 'c61'], 17: ['c24', 'c74', 'c94'], 18: ['c4', 'c28', 'c83'], 19: ['c18', 'c41', 'c60'], 20: ['c0', 'c27', 'c52'], 21: ['c33', 'c59', 'c93'], 22: ['c30', 'c75', 'c82'], 23: ['c10', 'c34', 'c77'], 24: ['c40', 'c55', 'c67'], 25: ['c23', 'c32', 'c56'], 26: ['c9', 'c22', 'c54'], 27: ['c16', 'c44', 'c88'], 28: ['c49', 'c63', 'c81'], 29: ['c37', 'c39', 'c87'], 30: ['c69', 'c74', 'c91'], 31: ['c11', 'c12', 'c26'], 32: ['c6', 'c50', 'c78'], 33: ['c1', 'c13', 'c38'], 34: ['c41', 'c84', 'c90'], 35: ['c35', 'c36', 'c66'], 36: ['c7', 'c30', 'c76'], 37: ['c58', 'c70', 'c75'], 38: ['c3', 'c18', 

In [9]:
# load used readouts
readout_usage_file = os.path.join(library_folder, 'readout_usage.pkl')
readout_dict = pickle.load(open(readout_usage_file, 'rb'))
len(readout_dict['c'])

95

## bit_2_readout

In [10]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
dtype_dict = {_v:_k for _k,_v in ia.classes._allowed_kwds.items()}

In [11]:
for _dtype, _records in readout_dict.items():
    
    bit_readout_df = pd.DataFrame([[_i+1, _r.id] for _i, _r in enumerate(_records)], columns=['Bit','ReadoutName'])
    save_filename = os.path.join(save_folder, f'CTP13-{os.path.basename(library_folder)}_{dtype_dict[_dtype]}_readouts.csv')
    if not os.path.exists(save_filename):
        print(save_filename)
        bit_readout_df.to_csv(save_filename, index=False)

In [12]:
bit_readout_df

,Bit,ReadoutName
0,1,NDB_376
1,2,NDB_578
2,3,NDB_400
3,4,NDB_533
4,5,NDB_568
...,...,...
90,91,NDB_592
91,92,NDB_368
92,93,NDB_436
93,94,NDB_629


# Codebook

In [13]:
# generate the codebook
codebook = pd.DataFrame(columns=['name','id']+[_r.id for _r in readout_dict['c']])
# decide region ids first
codebook['id'] = sorted(list(region_2_probeInfo.keys()), key=lambda v:int(v))
# region names
codebook['name'] = [region_2_probeInfo[str(_gene)] for _gene in codebook['id']]
# code
max_bit = np.max([np.max([int(_b[1:]) for _b in _bits]) for _gname, _bits in region_2_readout_dict.items()]) + 1
for _gene in codebook['id']:
    _bits = region_2_readout_dict[int(_gene)]
    binary_code = []
    for _i in range(max_bit):
        if f"c{_i}" in _bits:
            binary_code.append(1)
        else:
            binary_code.append(0)
    #print(_gene)
    codebook.loc[codebook['id']==_gene, codebook.columns[2:]] = binary_code
    #codebook.loc[codebook['name']==_gene,'id'] = int(_bits[0].split('u')[1])
# add chromosome info
codebook['chr'] = [_n.split(':')[0] for _n in codebook['name']]
for _chr in np.unique(codebook['chr']):
    _chr_codebook = codebook.loc[codebook['chr']==_chr]
    _chr_reg_info = [np.mean([int(_loc) for _loc in _n.split(':')[1].split('-')]) for _n in _chr_codebook['name']]
    _chr_orders = np.array(_chr_reg_info).argsort().argsort().astype(np.int32)
    codebook.loc[codebook['chr']==_chr, 'chr_order'] = _chr_orders

In [14]:
codebook

,name,id,NDB_376,NDB_578,NDB_400,NDB_533,NDB_568,NDB_464,NDB_466,NDB_431,...,NDB_608,NDB_460,NDB_563,NDB_592,NDB_368,NDB_436,NDB_629,NDB_604,chr,chr_order
0,1:9627926-9637875,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.0
1,1:9799472-9811359,2,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1.0
2,1:20920666-20929358,3,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,2.0
3,1:21522568-21534512,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3.0
4,1:21732182-21745770,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961,X:101961397-101974103,987,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,X,2.0
962,X:136808741-136818926,988,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,3.0
963,X:140517829-140527051,989,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,4.0
964,X:167157164-167167452,990,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,5.0


In [15]:
# sanity check
codebook.values[:,2:-2].sum(1)

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [16]:
# Save
save_filename = os.path.join(save_folder, f'CTP13-{os.path.basename(library_folder)}.csv')
if os.path.exists(save_filename):
    print(save_filename)
    codebook.to_csv(save_filename, index=None)

\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\Summary_tables\CTP13-mop_SE_1000.csv


# Add blank

In [20]:
def convert_bits_to_matrix(bits):
    _matrix = np.ones([len(bits), np.max(bits)+1], dtype=np.int32) * -1
    for _i, _b in enumerate(bits):
        _matrix[_i, _b] = 1
    return _matrix

In [21]:
on_num = 3
total_num = len(readout_dict['c'])

encoding_file = os.path.join(pool_folder, f'covering_v_{total_num}_k_{on_num}_t_2.txt')
loaded_barcodes = []

with open(encoding_file, 'r') as _handle:
    # start reading
    _lines = _handle.read().split('\n')
    for _line in _lines:
        _bits = [int(_i)-1 for _i in _line.split(' ') if len(_i) > 0]
        loaded_barcodes.append(_bits)

loaded_barcodes = np.array(loaded_barcodes, dtype=np.int32)
print(np.shape(loaded_barcodes))

# remove redundant barcodes
kept_barcodes = loaded_barcodes.copy()
kept_inds = np.zeros(len(kept_barcodes))
while np.sum(kept_inds) != len(kept_inds):
    kept_inds = np.ones(len(kept_barcodes), dtype=bool)
    _kept_mat = convert_bits_to_matrix(kept_barcodes)
    _dists = (total_num - (_kept_mat @ _kept_mat.T)) / 2
    _x, _y = np.where((_dists > 0) * (_dists <= 3))
    if len(_x) > 0:
        kept_inds[_x[0]] = False
        kept_barcodes = kept_barcodes[kept_inds]
        print(np.sum(kept_inds), kept_inds[_x[0]])
print(kept_barcodes.shape)

(1489, 3)
1488 False
1487 False
(1487, 3)


In [22]:
used_barcodes = []
for _reg, _code in region_2_readout_dict.items():
    _barcode = np.array([int(_c.split('c')[1]) for _c in _code])
    used_barcodes.append(_barcode)
used_barcodes = np.array(used_barcodes)

free_barcodes = []
for _barcode in kept_barcodes:
    if (used_barcodes == _barcode).all(1).any():
        continue
    else:
        free_barcodes.append(_barcode)

In [39]:
num_blank = 99

num_bits = len(readout_dict['c'])

blank_bit_usage = np.zeros(num_bits)
exist_bit_usage = np.zeros(num_bits)
for _b in used_barcodes:
    exist_bit_usage[_b] += 1
# init 
sel_blank_barcodes = []
# shuffle free barcodes
#np.random.shuffle(free_barcodes)
cand_barcodes = [_b for _b in free_barcodes]
while len(sel_blank_barcodes) < num_blank:
    # for each cand bit, estimate bit usage
    _bit_usages = np.array([exist_bit_usage[_c].sum() + blank_bit_usage[_c].sum() for _c in cand_barcodes])
    #_blank_bit_usages = np.array([blank_bit_usage[_c].sum() for _c in cand_barcodes])
    # randomly select one of the smallest used bits
    _min_inds = np.where(_bit_usages == np.min(_bit_usages))[0]
    _sel_min_ind = _min_inds[np.random.randint(len(_min_inds),)]
    # append
    _sel_barcode = cand_barcodes[_sel_min_ind].copy()
    sel_blank_barcodes.append(_sel_barcode)
    # pop this candidate
    cand_barcodes.pop(_sel_min_ind)
    # update usage
    blank_bit_usage[_sel_barcode] += 1
    exist_bit_usage[_sel_barcode] += 1
    

## Save blank

In [41]:
save_filename = os.path.join(save_folder, 'blank_barcodes.npy')
print(save_filename)
if not os.path.exists(save_filename):
    np.save(save_filename, np.array(sel_blank_barcodes))


\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\Summary_tables\blank_barcodes.npy


In [50]:
# generate the codebook_blank
codebook_blank = pd.DataFrame(columns=['name','id']+[_r.id for _r in readout_dict['c']])
max_bit = np.max([np.max([int(_b[1:]) for _b in _bits]) for _gname, _bits in region_2_readout_dict.items()]) + 1
# decide region ids first
_id_start = np.max((codebook['id']).values.astype(np.int32)) + 1
codebook_blank['id'] = np.arange(_id_start, _id_start+len(sel_blank_barcodes))
# decide region names
codebook_blank['name'] = [f'B:{_i + 100}-{_i+1 + 100}' for _i in range(len(sel_blank_barcodes))]
for _i, _name in enumerate(codebook_blank['name']):
    _barcode = sel_blank_barcodes[_i]
    binary_code = np.zeros(max_bit, dtype=np.int32)
    binary_code[_barcode] = 1
    codebook_blank.loc[codebook_blank['name']==_name, codebook_blank.columns[2:]] = binary_code
# add chromosome info
codebook_blank['chr'] = [_n.split(':')[0] for _n in codebook_blank['name']]
for _chr in np.unique(codebook_blank['chr']):
    _chr_codebook_blank = codebook_blank.loc[codebook_blank['chr']==_chr]
    _chr_reg_info = [np.mean([int(_loc) for _loc in _n.split(':')[1].split('-')]) for _n in _chr_codebook_blank['name']]
    _chr_orders = np.array(_chr_reg_info).argsort().argsort().astype(np.int32)
    codebook_blank.loc[codebook_blank['chr']==_chr, 'chr_order'] = _chr_orders

In [51]:
# Merge blank with codebook
codebook_w_blank = pd.concat([codebook,codebook_blank], ignore_index=True)
# sanity check
np.unique(codebook_w_blank.values[:,2:-2].astype(int).sum(1))

array([3])

In [52]:
codebook_w_blank

,name,id,NDB_376,NDB_578,NDB_400,NDB_533,NDB_568,NDB_464,NDB_466,NDB_431,...,NDB_608,NDB_460,NDB_563,NDB_592,NDB_368,NDB_436,NDB_629,NDB_604,chr,chr_order
0,1:9627926-9637875,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.0
1,1:9799472-9811359,2,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1.0
2,1:20920666-20929358,3,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,2.0
3,1:21522568-21534512,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,3.0
4,1:21732182-21745770,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1060,B:194-195,1086,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,B,94.0
1061,B:195-196,1087,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,B,95.0
1062,B:196-197,1088,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,B,96.0
1063,B:197-198,1089,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,B,97.0


In [53]:
# save codebook
save_filename = os.path.join(save_folder, f'CTP13-{os.path.basename(library_folder)}_w_blanks.csv')
if not os.path.exists(save_filename):
    print(save_filename)
    codebook_w_blank.to_csv(save_filename,index=None)

\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-13_brain-super-enhancers\Summary_tables\CTP13-mop_SE_1000_w_blanks.csv
